In [ ]:
import matplotlib
# %matplotlib inline
# print("BACKEND: ", matplotlib.get_backend())
# if matplotlib.get_backend() != "macosx":
#   print("Changing backend to macosx")
#   matplotlib.use('macosx')


In [ ]:

import matplotlib.pyplot as plt
%matplotlib inline

# matplotlib.use("Agg")

from ase.visualize import view

from ase import Atoms
from ase.build import bulk
from ase.io import read, write
from agox.databases import Database
from agox.environments import Environment
from agox.utils.graph_sorting import Analysis

In [ ]:
from agox.models.descriptors.fingerprint import Fingerprint
from agox.models.descriptors import Voronoi
import numpy as np

template = Atoms("", cell=np.eye(3) * 3.567, pbc=True)
diamond = bulk("C", "diamond", a=3.567)  # Lattice constant for diamond cubic carbon
confinement_cell = template.cell.copy()
confinement_corner = np.array([0, 0, 0])
environment = Environment(
    template=template,
    symbols="C8",
    confinement_cell=confinement_cell,
    confinement_corner=confinement_corner,
    box_constraint_pbc=[True, True, True],  # Confinement is periodic in all directions.
)
descriptor = Fingerprint(environment=environment)
graph_descriptor = Voronoi(
    covalent_bond_scale_factor=1.3, n_points=8, angle_from_central_atom=20, environment=environment
)


In [ ]:
graph_sorting = Analysis(descriptor=graph_descriptor, directories=["."], sample_size=40)

In [ ]:
# structures = read("DMACE/D3-5/all.traj", index=":")
structures = read("all.traj", index=":")


In [ ]:
unique, count = graph_sorting.sort_structures(structures=structures)

In [ ]:
# Calculate energies per atom for each unique structure
energies_per_atom = [structure.get_potential_energy() / len(structure) for structure in unique]
delta_en_per_atom = energies_per_atom - np.min(energies_per_atom)

# Plot the number of unique structures vs the energies per atom
plt.figure(figsize=(10, 6))
plt.bar(delta_en_per_atom, count)
plt.xlabel('Unique Structure Index')
plt.ylabel('Energy per Atom (eV)')
plt.title('Number of Unique Structures vs Energies per Atom')
plt.show()

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
pca = PCA(n_components=2)

In [ ]:
X_reduced = pca.fit_transform(np.squeeze([arr for arr in descriptor.get_features(unique)]))

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=delta_en_per_atom, cmap="viridis")

In [ ]:
plt.show()